# HW Test Case
Examine a handful of HW measurements in detail.

HW10 report a 75km/s dispersion between MgII_cc and [OIII].
But they surely can't know this at z>1, right?

In [51]:
# imports
from linetools.spectra import io as lsio
import qpq_z as qpqz
from xastropy.sdss import qso as sdssqso
import hewittwildz as hw10z
from astropy.coordinates import SkyCoord
from xastropy.sdss import quasars as sdssq

## Setup

In [2]:
# Load good MgII list from JFH
reload(qpqz)
gqsos, z_O2, z_line = qpqz.parse_o2_zstruct('MgII')

In [8]:
# Load HW Table from Vizier
#  Could grab the Table from MNRAS (I think)
hw10_tab = Table.read('hw10_tab4.fits.gz')
hw10_tab[0:3]

SDSS,z,e_z,Det,z1,n_z,Plate,MJD,Fiber,Sloan,DR7,RAJ2000,DEJ2000
,,,,,,,d,,,,deg,deg
string152,float32,float32,int16,float32,uint8,int16,int32,int16,string40,string24,float64,float64
J000006.53+003055.2,1.8232,0.001,-1,nan,3,685,52203,467,Sloan,DR7,0.02723,0.51534
J000008.13+001634.6,1.8363,0.0006,-1,nan,3,685,52203,470,Sloan,DR7,0.0339,0.2763
J000009.26+151754.5,1.1974,0.0004,0,1.2,2,751,52251,354,Sloan,DR7,0.0386,15.29848


In [164]:
# Good redshifts for zMgII analysis
zMgII = np.where(np.abs(hw10_tab['z']-1.4) < 0.3)[0]

In [52]:
# Load DR7
sdss_dr7 = sdssq.SdssQuasars(verbose=False)

## Try First One

In [92]:
idx1=1
spec_file = os.getenv('DROPBOX_DIR')+'/O2_zsys/sdss_boss_o2/spectra/'+gqsos[idx1]['SPEC_FILE']
print(spec_file)
print(z_O2[idx1])
print(gqsos['PLATE'][idx1], gqsos['FIBERID'][idx1])

/u/xavier/Dropbox//O2_zsys/sdss_boss_o2/spectra/SDSSJ003153.84+005916.2.fits
1.36684204789
(3587, 552)


In [66]:
# HW
idx_HW1 = np.where( (np.abs(hw10_tab['RAJ2000']-gqsos['PLUG_RA'][idx1]) < 1e-4) &
                  (np.abs(hw10_tab['DEJ2000']-gqsos['PLUG_DEC'][idx1]) < 1e-4))[0]
hw10_tab[idx_HW1]

SDSS,z,e_z,Det,z1,n_z,Plate,MJD,Fiber,Sloan,DR7,RAJ2000,DEJ2000
,,,,,,,d,,,,deg,deg
string152,float32,float32,int16,float32,uint8,int16,int32,int16,string40,string24,float64,float64
J003153.84+005916.1,1.3659,0.0005,-1,nan,3,391,51782,603,Sloan,DR7,7.97433,0.98783


In [70]:
# SDSSQSO Class
qso1 = sdss_dr7[hw10_tab['Plate'][idx_HW1], hw10_tab['Fiber'][idx_HW1]]
qso1.load_spec()

SdssQso: Loading spectrum from /u/xavier/SDSS//DR7_QSO/spectro/1d_26/0391/1d/spSpec-51782-0391-603.fit


In [162]:
# Run HW10 algorithm
reload(hw10z)
qso1.spec.z = qso1.z
print('z_init = {:g}'.format(qso1.z))
hwz1 = hw10z.HewittWildz(qso1.spec)
hwz1.run(debug=True,niter=2,msk_for_mgii=True)
hwz1

z_init = 1.36326
nlag=100, olag = 3.84965
nlag=80, olag = 5.22534
nlag=60, olag = 5.97515
nlag=40, olag = 5.57626
nlag=20, olag = 4.69399
zguess=1.36326, zfin=1.36581
nlag=100, olag = 0.498189
nlag=80, olag = 1.37875
nlag=60, olag = 1.63502
nlag=40, olag = 0.692836
nlag=20, olag = 0.14168
zguess=1.36581, zfin=1.36589


{'wvmax': <Quantity 3299.2817036109304 Angstrom>, 'zguess': 1.3658140213124523, 'wvmin': <Quantity 2200.489996727403 Angstrom>, 'zinit': 1.36326, u'zfin': 1.3658911926764696, u'cc_max': 0.63684254459078538}

In [163]:
# Offsets
HWvdiff1 = 3e5*(hwz1.z_dict['zfin']-hw10_tab['z'][idx_HW1[0]])/(hwz1.z_dict['zfin']+1)
print('HW10 vs JXP_HW = {:g}'.format(HWvdiff1))
o2_vdiff1 = 3e5*(hwz1.z_dict['zfin']-z_O2[idx1])/(hwz1.z_dict['zfin']+1)
print('[OII] vs JXP_HW = {:g}'.format(o2_vdiff1))

HW10 vs JXP_HW = -1.12182
[OII] vs JXP_HW = -120.57


## Try Another

In [209]:
# HW
idx_HW2 = zMgII[10]
hw10_tab[idx_HW2+np.arange(1)]

SDSS,z,e_z,Det,z1,n_z,Plate,MJD,Fiber,Sloan,DR7,RAJ2000,DEJ2000
,,,,,,,d,,,,deg,deg
string152,float32,float32,int16,float32,uint8,int16,int32,int16,string40,string24,float64,float64
J000053.09-003712.8,1.3247,0.0005,-1,nan,3,387,51791,169,Sloan,DR7,0.22122,-0.62024


In [210]:
# SDSSQSO Class
qso2 = sdss_dr7[hw10_tab['Plate'][idx_HW2], hw10_tab['Fiber'][idx_HW2]]
qso2.load_spec()

SdssQso: Loading spectrum from /u/xavier/SDSS//DR7_QSO/spectro/1d_26/0387/1d/spSpec-51791-0387-169.fit


In [264]:
# Run HW10 algorithm
reload(hw10z)
qso2.spec.z = qso2.z
print('z_init = {:g}'.format(qso2.z))
hwz2 = hw10z.HewittWildz(qso2.spec)
hwz2.run(debug=True,niter=2,msk_for_mgii=True)
hwz2

z_init = 1.3208
nlag=100, olag = 5.86136
nlag=80, olag = 7.48997
nlag=60, olag = 8.98672
nlag=40, olag = 9.96489
nlag=20, olag = 10.9397
zguess=1.3208, zfin=1.32665
nlag=100, olag = -1.90623
nlag=80, olag = -2.37133
nlag=60, olag = -2.29385
nlag=40, olag = -1.14942
nlag=20, olag = 0.00739869
zguess=1.32665, zfin=1.32665


{'wvmax': <Quantity 3299.664782327628 Angstrom>, 'zguess': 1.3266452940333897, 'wvmin': <Quantity 2200.238812991854 Angstrom>, 'zinit': 1.3207999, u'zfin': 1.3266492572775639, u'cc_max': 0.68035952082379603}

In [265]:
hwz2.qa_plot(hw10_tab['SDSS'][idx_HW2], ztwo=hw10_tab['z'][idx_HW2], 
             templ_xymnx=dict(xmin=2700.,xmax=2900.,ymin=-2,ymax=8))

Wrote: J000053.09-003712.8.pdf


In [217]:
# Offsets
HWvdiff2 = 3e5*(hwz2.z_dict['zfin']-hw10_tab['z'][idx_HW2])/(hwz2.z_dict['zfin']+1)
print('HW10 vs JXP_HW = {:g}'.format(HWvdiff2))
#o2_vdiff2 = 3e5*(hwz2.z_dict['zfin']-z_O2[idx2])/(hwz2.z_dict['zfin']+1)
#print('[OII] vs JXP_HW = {:g}'.format(o2_vdiff2))

HW10 vs JXP_HW = 251.339


## Try one more

In [257]:
# HW
idx_HW3 = zMgII[1533]
hw10_tab[idx_HW3+np.arange(1)]

SDSS,z,e_z,Det,z1,n_z,Plate,MJD,Fiber,Sloan,DR7,RAJ2000,DEJ2000
,,,,,,,d,,,,deg,deg
string152,float32,float32,int16,float32,uint8,int16,int32,int16,string40,string24,float64,float64
J015248.43+011442.6,1.1715,0.0005,-1,nan,3,402,51793,521,Sloan,DR7,28.2018,1.24519


In [258]:
# SDSSQSO Class
qso3 = sdss_dr7[hw10_tab['Plate'][idx_HW3], hw10_tab['Fiber'][idx_HW3]]
qso3.load_spec()

SdssQso: Loading spectrum from /u/xavier/SDSS//DR7_QSO/spectro/1d_26/0402/1d/spSpec-51793-0402-521.fit


In [266]:
# Run HW10 algorithm
reload(hw10z)
qso3.spec.z = qso3.z
print('z_init = {:g}'.format(qso3.z))
hwz3 = hw10z.HewittWildz(qso3.spec)
hwz3.run(debug=True,niter=2,msk_for_mgii=True)
hwz3

z_init = 1.17133
nlag=100, olag = 3.68214
nlag=80, olag = 3.80046
nlag=60, olag = 3.2258
nlag=40, olag = 3.15986
nlag=20, olag = 3.39952
zguess=1.17133, zfin=1.17303
nlag=100, olag = 1.13813
nlag=80, olag = 0.950661
nlag=60, olag = 0.0776723
nlag=40, olag = -0.306479
nlag=20, olag = -0.401356
zguess=1.17303, zfin=1.17283


{'wvmax': <Quantity 3299.3956139660027 Angstrom>, 'zguess': 1.1730294255880258, 'wvmin': <Quantity 2200.059329706267 Angstrom>, 'zinit': 1.17133, u'zfin': 1.172828626790637, u'cc_max': 0.64399763654730102}

In [267]:
hwz3.qa_plot(hw10_tab['SDSS'][idx_HW3], ztwo=hw10_tab['z'][idx_HW3],
             templ_xymnx=dict(xmin=2700.,xmax=2900.,ymin=-2,ymax=6))

Wrote: J015248.43+011442.6.pdf


In [260]:
# Offsets
HWvdiff3 = 3e5*(hwz3.z_dict['zfin']-hw10_tab['z'][idx_HW3])/(hwz3.z_dict['zfin']+1)
print('HW10 vs JXP_HW = {:g}'.format(HWvdiff3))

HW10 vs JXP_HW = 183.446


In [186]:
hwz3.chk_spec_templ()